In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square
from netgen.geom2d import SplineGeometry

geo = SplineGeometry()


In [2]:
#finally found a way to generate a regular mesh with the boundary condition names set correctly
# just remove the maxh value from unit_square.GenerateMesh, then in Netgen refine it a couple times and save it.
# it's unfortunate that we can load geometry files but not save them.

mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))#maxh=0.25))
#print(unit_square.SegmentData())
#mesh = Mesh('dow_test7.vol')
#mesh.Boundaries('top')

In [3]:
mesh.nv, mesh.ne

(25, 32)

In [4]:

fes = H1(mesh, order=1, dirichlet="bottom|top|left|right")
fes.ndof  # number of unknowns in this space


25

In [5]:
u = fes.TrialFunction()
v = fes.TestFunction()
gfu = GridFunction(fes)

In [6]:
a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI(grad(u)*grad(v))

In [7]:
a.Assemble()

In [8]:
f = LinearForm(fes)
f += SymbolicLFI(x*x*v)
f.Assemble()

In [9]:
print(f.vec)

 0.000244141
 0.00919596
 0.0171712
 8.13802e-05
 0.0094401
 0.016276
 0.000325521
 0.0263672
 0.00683594
 0.00292969
 0.00455729
 0.000325521
 0.0198568
 0.0263672
 0.0358073
 0.0263672
 0.0159505
 0.000325521
 0.0016276
 0.016276
 0.0358073
 0.00455729
 0.0358073
 0.016276
 0.00455729




I tried to get Ngsolve to generate a matrix similar to the one in Claes Johnson 1.4, but it always puts in rows for the boundary elements and the diagonal elements have a different pattern

In [10]:
print(a.mat)

Row 0:   0: 1
Row 1:   1: 1
Row 2:   2: 1
Row 3:   3: 1
Row 4:   4: 2
Row 5:   5: 4
Row 6:   6: 2
Row 7:   7: 2
Row 8:   8: 2
Row 9:   0: -0.5   4: -0.5   9: 2
Row 10:   0: 0   5: 0   9: -1   10: 4
Row 11:   0: -0.5   6: -0.5   10: -1   11: 2
Row 12:   1: -0.5   4: -0.5   12: 2
Row 13:   1: -0.5   7: -0.5   12: 0   13: 2
Row 14:   2: 0   5: 0   14: 4
Row 15:   2: -0.5   7: -0.5   14: -1   15: 2
Row 16:   2: -0.5   8: -0.5   14: -1   16: 2
Row 17:   3: -0.5   6: -0.5   17: 2
Row 18:   3: -0.5   8: -0.5   17: 0   18: 2
Row 19:   4: -1   5: -1   9: 0   10: -1   19: 4
Row 20:   4: 0   7: 0   12: -1   13: -1   19: -1   20: 4
Row 21:   5: -1   6: -1   10: -1   11: 0   21: 4
Row 22:   5: -1   7: -1   14: -1   15: 0   19: 0   20: -1   22: 4
Row 23:   5: -1   8: -1   14: -1   16: 0   21: 0   23: 4
Row 24:   6: 0   8: 0   17: -1   18: -1   21: -1   23: -1   24: 4



In [11]:
#gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse='umfpack')*f.vec
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse='sparsecholesky')*f.vec

In [12]:
Draw(gfu)

In [13]:
print(gfu.vec)

       0
       0
       0
       0
       0
 0.0202637
       0
       0
       0
       0
 0.00756255
       0
       0
       0
 0.0187233
       0
       0
       0
       0
 0.0157064
 0.0187233
 0.00998651
 0.0233794
 0.0157064
 0.00756255


